---
title: Assignment 04 AI Prompts
author:
  - name: Ava Godsy
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
number-sections: true
date: today
date-modified: today
date-format: long
format:
  html:
    theme: cerulean
    toc: true
    toc-depth: 2
  docx: default
  pdf: default
execute:
  echo: false
  eval: false
  freeze: auto
---

# Feature Engineering
## AI Prompt
create a dataframe using python with the following: from dataframe df Drop rows with missing values in the target variable and key features.
By now you are already familiar with the code and the data. Based on your understanding please choose any 3 (my code output has 10) variables as: 1. choose variables: MIN_YEARS_EXPERIENCE , MAX_YEARS_EXPERIENCE, SALARY_FROM, SALARY_TO, MSA_NAME, REMOTE_TYPE_NAME 2. Your dependent variable (y) is SALARY.
Convert MSA_NAME, REMOTE_TYPE_NAME into numerical representations using StringIndexer and OneHotEncoder.
Assemble features into a single vector using VectorAssembler.
Split the data into training and testing sets.
You can use pipeline to do the above steps in one go.
Create a new column MIN_YEARS_EXPERIENCE_SQ by squaring the MIN_YEARS_EXPERIENCE column.
Assemble the polynomial features into a new vector column features_poly using VectorAssembler.
Show the final structure of the DataFrame with the new features.

## Claude Response
I'll create a Python script that performs all these data preprocessing steps. Since you mentioned you're working with a DataFrame df, I'll create a comprehensive solution using PySpark (based on your mention of StringIndexer, OneHotEncoder, and VectorAssembler).

In [ ]:
#|echo: false
#|eval: false
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.sql.functions import col, pow as spark_pow
from pyspark.ml.regression import LinearRegression

# Initialize Spark Session (if not already created)
spark = SparkSession.builder.appName("SalaryPrediction").getOrCreate()

# Assuming df is your existing DataFrame
# Step 1: Drop rows with missing values in target and key features
selected_columns = ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE', 'SALARY_FROM', 
                   'MSA_NAME', 'REMOTE_TYPE_NAME', 'SALARY']

df_clean = df.select(selected_columns).dropna()

print("Original DataFrame count:", df.count())
print("Cleaned DataFrame count:", df_clean.count())
print("\nCleaned DataFrame Schema:")
df_clean.printSchema()

# Step 2: Create squared feature for MIN_YEARS_EXPERIENCE
df_clean = df_clean.withColumn('MIN_YEARS_EXPERIENCE_SQ', 
                               spark_pow(col('MIN_YEARS_EXPERIENCE'), 2))

print("\nDataFrame with squared feature:")
df_clean.show(5)

# Step 3: Create Pipeline for encoding and feature assembly

# StringIndexer for categorical variables
msa_indexer = StringIndexer(inputCol='MSA_NAME', 
                            outputCol='MSA_NAME_INDEX',
                            handleInvalid='keep')

remote_indexer = StringIndexer(inputCol='REMOTE_TYPE_NAME', 
                               outputCol='REMOTE_TYPE_NAME_INDEX',
                               handleInvalid='keep')

# OneHotEncoder for categorical variables
msa_encoder = OneHotEncoder(inputCol='MSA_NAME_INDEX', 
                           outputCol='MSA_NAME_VEC',
                           dropLast=True)

remote_encoder = OneHotEncoder(inputCol='REMOTE_TYPE_NAME_INDEX', 
                              outputCol='REMOTE_TYPE_NAME_VEC',
                              dropLast=True)

# VectorAssembler for basic features
feature_cols = ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE', 'SALARY_FROM',
               'MSA_NAME_VEC', 'REMOTE_TYPE_NAME_VEC']

assembler = VectorAssembler(inputCols=feature_cols, 
                            outputCol='features',
                            handleInvalid='keep')

# VectorAssembler for polynomial features
poly_feature_cols = ['MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ', 
                    'MAX_YEARS_EXPERIENCE', 'SALARY_FROM',
                    'MSA_NAME_VEC', 'REMOTE_TYPE_NAME_VEC']

poly_assembler = VectorAssembler(inputCols=poly_feature_cols, 
                                outputCol='features_poly',
                                handleInvalid='keep')

# Create Pipeline
pipeline = Pipeline(stages=[
    msa_indexer,
    remote_indexer,
    msa_encoder,
    remote_encoder,
    assembler,
    poly_assembler
])

# Fit and transform the data
pipeline_model = pipeline.fit(df_clean)
df_transformed = pipeline_model.transform(df_clean)

print("\nTransformed DataFrame with all features:")
df_transformed.select('MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ', 
                     'MAX_YEARS_EXPERIENCE', 'SALARY_FROM', 
                     'MSA_NAME', 'REMOTE_TYPE_NAME', 'SALARY',
                     'features', 'features_poly').show(5, truncate=False)

# Step 4: Split data into training and testing sets
train_data, test_data = df_transformed.randomSplit([0.8, 0.2], seed=42)

print(f"\nTraining set count: {train_data.count()}")
print(f"Testing set count: {test_data.count()}")

# Step 5: Show final structure
print("\n=== FINAL DATAFRAME STRUCTURE ===")
df_transformed.printSchema()

print("\n=== SAMPLE OF FINAL DATA ===")
df_transformed.select('MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ',
                     'MAX_YEARS_EXPERIENCE', 'SALARY_FROM',
                     'SALARY', 'features_poly').show(10)

# Display feature statistics
print("\n=== FEATURE STATISTICS ===")
df_transformed.select('MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ',
                     'MAX_YEARS_EXPERIENCE', 'SALARY_FROM', 
                     'SALARY').describe().show()

# Optional: Show unique values in categorical columns
print("\n=== CATEGORICAL VARIABLE COUNTS ===")
print("MSA_NAME unique values:", df_clean.select('MSA_NAME').distinct().count())
print("REMOTE_TYPE_NAME unique values:", df_clean.select('REMOTE_TYPE_NAME').distinct().count())

# Save transformed data for future use (optional)
# df_transformed.write.parquet("transformed_salary_data.parquet", mode='overwrite')

print("\n✓ Data preprocessing pipeline completed successfully!")
print("✓ Ready for model training with 'features_poly' as input and 'SALARY' as target")

I've created a comprehensive PySpark data preprocessing pipeline that accomplishes all your requirements:
Key Features:

Data Cleaning: Drops rows with missing values in the selected features and target variable (SALARY)
Selected Features:

MIN_YEARS_EXPERIENCE
MAX_YEARS_EXPERIENCE
SALARY_FROM
MSA_NAME (categorical)
REMOTE_TYPE_NAME (categorical)


Polynomial Feature: Creates MIN_YEARS_EXPERIENCE_SQ by squaring MIN_YEARS_EXPERIENCE
Pipeline Components:

StringIndexer for both categorical variables
OneHotEncoder for converting indexed categories to vectors
VectorAssembler for basic features → 'features' column
VectorAssembler for polynomial features → 'features_poly' column


Train-Test Split: 80-20 split with a random seed for reproducibility
Final Output: Shows the complete structure with both 'features' and 'features_poly' vectors

The pipeline approach ensures all transformations are applied consistently to both training and testing data. The 'features_poly' column includes the squared term and is ready for polynomial regression modeling!

# Train/Test Split
## Prompt
(This is a continuation of the previous prompt)

Perform a random split of the data into training and testing sets.
Set a random seed for reproducibility.
You can choose a number for splitting to your liking, justify your choice.

## Claude Response


In [ ]:
#|echo: false
#|eval: false
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.sql.functions import col, pow as spark_pow
from pyspark.ml.regression import LinearRegression

# Initialize Spark Session (if not already created)
spark = SparkSession.builder.appName("SalaryPrediction").getOrCreate()

# Assuming df is your existing DataFrame
# Step 1: Drop rows with missing values in target and key features
selected_columns = ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE', 'SALARY_FROM', 
                   'MSA_NAME', 'REMOTE_TYPE_NAME', 'SALARY']

df_clean = df.select(selected_columns).dropna()

print("Original DataFrame count:", df.count())
print("Cleaned DataFrame count:", df_clean.count())
print("\nCleaned DataFrame Schema:")
df_clean.printSchema()

# Step 2: Create squared feature for MIN_YEARS_EXPERIENCE
df_clean = df_clean.withColumn('MIN_YEARS_EXPERIENCE_SQ', 
                               spark_pow(col('MIN_YEARS_EXPERIENCE'), 2))

print("\nDataFrame with squared feature:")
df_clean.show(5)

# Step 3: Create Pipeline for encoding and feature assembly

# StringIndexer for categorical variables
msa_indexer = StringIndexer(inputCol='MSA_NAME', 
                            outputCol='MSA_NAME_INDEX',
                            handleInvalid='keep')

remote_indexer = StringIndexer(inputCol='REMOTE_TYPE_NAME', 
                               outputCol='REMOTE_TYPE_NAME_INDEX',
                               handleInvalid='keep')

# OneHotEncoder for categorical variables
msa_encoder = OneHotEncoder(inputCol='MSA_NAME_INDEX', 
                           outputCol='MSA_NAME_VEC',
                           dropLast=True)

remote_encoder = OneHotEncoder(inputCol='REMOTE_TYPE_NAME_INDEX', 
                              outputCol='REMOTE_TYPE_NAME_VEC',
                              dropLast=True)

# VectorAssembler for basic features
feature_cols = ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE', 'SALARY_FROM',
               'MSA_NAME_VEC', 'REMOTE_TYPE_NAME_VEC']

assembler = VectorAssembler(inputCols=feature_cols, 
                            outputCol='features',
                            handleInvalid='keep')

# VectorAssembler for polynomial features
poly_feature_cols = ['MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ', 
                    'MAX_YEARS_EXPERIENCE', 'SALARY_FROM',
                    'MSA_NAME_VEC', 'REMOTE_TYPE_NAME_VEC']

poly_assembler = VectorAssembler(inputCols=poly_feature_cols, 
                                outputCol='features_poly',
                                handleInvalid='keep')

# Create Pipeline
pipeline = Pipeline(stages=[
    msa_indexer,
    remote_indexer,
    msa_encoder,
    remote_encoder,
    assembler,
    poly_assembler
])

# Fit and transform the data
pipeline_model = pipeline.fit(df_clean)
df_transformed = pipeline_model.transform(df_clean)

print("\nTransformed DataFrame with all features:")
df_transformed.select('MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ', 
                     'MAX_YEARS_EXPERIENCE', 'SALARY_FROM', 
                     'MSA_NAME', 'REMOTE_TYPE_NAME', 'SALARY',
                     'features', 'features_poly').show(5, truncate=False)

# Step 4: Split data into training and testing sets
# Using 70-30 split for better evaluation capability
# Justification:
# - 70% training: Provides sufficient data for model to learn patterns
# - 30% testing: Larger test set gives more reliable performance metrics
# - Good balance for datasets with moderate size (thousands of records)
# - Seed=42 ensures reproducibility across runs
train_data, test_data = df_transformed.randomSplit([0.7, 0.3], seed=42)

print(f"\n=== DATA SPLIT SUMMARY ===")
print(f"Training set count: {train_data.count()} ({train_data.count()/df_transformed.count()*100:.1f}%)")
print(f"Testing set count: {test_data.count()} ({test_data.count()/df_transformed.count()*100:.1f}%)")
print(f"Random seed: 42 (for reproducibility)")
print("\nSplit Justification:")
print("• 70-30 split provides robust model evaluation")
print("• Larger test set (30%) improves confidence in performance metrics")
print("• Balanced approach for moderate-sized datasets")
print("• Alternative splits: 80-20 for large datasets, 60-40 for small datasets")

# Step 5: Show final structure
print("\n=== FINAL DATAFRAME STRUCTURE ===")
df_transformed.printSchema()

print("\n=== SAMPLE OF FINAL DATA ===")
df_transformed.select('MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ',
                     'MAX_YEARS_EXPERIENCE', 'SALARY_FROM',
                     'SALARY', 'features_poly').show(10)

# Display feature statistics
print("\n=== FEATURE STATISTICS ===")
df_transformed.select('MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ',
                     'MAX_YEARS_EXPERIENCE', 'SALARY_FROM', 
                     'SALARY').describe().show()

# Optional: Show unique values in categorical columns
print("\n=== CATEGORICAL VARIABLE COUNTS ===")
print("MSA_NAME unique values:", df_clean.select('MSA_NAME').distinct().count())
print("REMOTE_TYPE_NAME unique values:", df_clean.select('REMOTE_TYPE_NAME').distinct().count())

# Save transformed data for future use (optional)
# df_transformed.write.parquet("transformed_salary_data.parquet", mode='overwrite')

print("\n✓ Data preprocessing pipeline completed successfully!")
print("✓ Ready for model training with 'features_poly' as input and 'SALARY' as target")